In [5]:
import random
from names import get_full_name
from tqdm import tqdm
from scipy import stats

def expected_win_chance(elo_diff):
    return 1 / (1 + 10**(elo_diff/400))

def expected_draw_chance(elo_diff, elo_level):
    a,b,c = -0.07067227,  0.01541295,  9.57851891
    return a*elo_diff + b*elo_level + c

class competition:
    
    def __init__(self, nr_players):
        self.player_poule = [player(skill_level = stats.truncnorm((0 - 0.5) / 0.15, (1 - 0.5) / 0.15, loc=0.5, scale=0.15).rvs(1)[0]) for i in tqdm(range(nr_players))]
        
    def play_rand_matches(self, nr_of_games):
        for game in tqdm(range(nr_of_games)):
            players = random.sample(range(0, len(self.player_poule)), 2)
            
            m = match(self.player_poule[players[0]], self.player_poule[players[1]])
            m.simulate_match()
        
    def show_standings(self):
        sorted_poule = sorted(self.player_poule, key=lambda x: x.rating, reverse=True)
        print("leaderboard standings")
        print("------------------------------------")
        for p in sorted_poule[0:10]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), round(p.rating), p.skill_level))
        for p in sorted_poule[-11:-1]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), round(p.rating), p.skill_level))
        print("------------------------------------\n")
    

class match:
    
    def __init__(self, player1, player2, k_factor = 32):
        self.p1 = player1
        self.p2 = player2
        
        self.es_p1 = 1 / (1 + 10**((player2.rating - player1.rating) / 400))
        self.es_p2 = 1 / (1 + 10**((player1.rating - player2.rating) / 400))
    
    def simulate_match(self):

        win_chances = self.calc_odds(self.p1.skill_level, self.p2.skill_level)

        result_p1 = random.choices([1, 0.5, 0], weights = win_chances, k=1)[0]
        result_p2 = 1-result_p1
        
        self.p1.update_rating(self.es_p1, result_p1)
        self.p2.update_rating(self.es_p2, result_p2)

        self.p1.games_played += 1
        self.p2.games_played += 1

        self.p1.update_k_factor()
        self.p2.update_k_factor
        
        return result_p1

    def calc_odds(self, sl_p1, sl_p2):
        mapped_sl_p1 = 900 + sl_p1 * 2000
        mapped_sl_p2 = 900 + sl_p2 * 2000
        elo_diff = mapped_sl_p2 - mapped_sl_p1

        p1_win_odds = expected_win_chance(elo_diff)
        p2_win_odds = expected_win_chance(-elo_diff)
        draw_odds = expected_draw_chance(abs(elo_diff), max([mapped_sl_p1, mapped_sl_p2]))
        
        # Correct for imperfect draw odds function
        draw_odds = draw_odds if draw_odds <= min([p1_win_odds, p2_win_odds]) else min([p1_win_odds, p2_win_odds])*2

        p1_win_odds -= draw_odds/2
        p2_win_odds -= draw_odds/2

        return [p1_win_odds, draw_odds, p2_win_odds]



class player:
    
    def __init__(self, rating = 1900, skill_level = 0.5, name = "filler name"):
        self.name = name
        self.rating = rating
        self.skill_level = skill_level
        self.k_factor = 40
        self.games_played = 0
        self.has_hit_2400 = False

    
    def update_rating(self, expected_score, result):
        change = self.k_factor * (result - expected_score)
        self.rating += change

    def update_k_factor(self):
        if self.rating >= 2400:
            self.has_hit_2400 = True
        
        if self.games_played >= 30:
            if self.has_hit_2400:
                self.k_factor = 10
            else:
                self.k_factor = 20
        else:
            self.k_factor = 40
            
        



In [7]:
c = competition(2000)
c.show_standings()
c.play_rand_matches(10000000)
c.show_standings()

100%|██████████| 10000000/10000000 [05:08<00:00, 32451.25it/s]

leaderboard standings
------------------------------------
filler name           | 2210 | 0.703
filler name           | 2200 | 0.870
filler name           | 2199 | 0.792
filler name           | 2180 | 0.743
filler name           | 2177 | 0.717
filler name           | 2176 | 0.792
filler name           | 2171 | 0.705
filler name           | 2164 | 0.734
filler name           | 2164 | 0.754
filler name           | 2163 | 0.752
filler name           | 1671 | 0.324
filler name           | 1670 | 0.362
filler name           | 1667 | 0.318
filler name           | 1666 | 0.318
filler name           | 1664 | 0.308
filler name           | 1662 | 0.312
filler name           | 1661 | 0.324
filler name           | 1661 | 0.318
filler name           | 1648 | 0.277
filler name           | 1644 | 0.336
------------------------------------



In [10]:
# NOTES
# - gaat er vanuit dat iedereen tegen iedereen speelt
# - match simulatie gaat nog niet goed. de beste moeten meer winnen

# Map to 900-1900-2900, but FIDE rules don't allow elo to be below certain point (I think it is 1000 or 1200)

# K is the development coefficient.
# K = 40 for a player new to the rating list until he has completed events with at least 30 games
# K = 20 as long as a player's rating remains under 2400.
# K = 10 once a player's published rating has reached 2400 and remains at that level subsequently, even if the rating drops below 2400.
# K = 40 for all players until their 18th birthday, as long as their rating remains under 2300.


In [4]:
name = "jeroen kortus"
rating = 900.99
skill_level = 0.9763224
print("{} | {:4d} | {:.3f}".format(name.ljust(21), round(rating), skill_level))

jeroen kortus         |  901 | 0.976
